# Importando e tratando os dados

In [2]:
!pip install kagglehub

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("asaniczka/tmdb-movies-dataset-2023-930k-movies")

print("Path to dataset files:", path)

100%|██████████| 219M/219M [00:02<00:00, 110MB/s] 

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/619


In [4]:
import pandas as pd
colunas_escolhidas = ["id", "title", "vote_average", "vote_count", "release_date", "revenue", "runtime", "adult", "budget", "overview", "popularity", "genres", "keywords"]
dados = pd.read_csv("/root/.cache/kagglehub/datasets/asaniczka/tmdb-movies-dataset-2023-930k-movies/versions/619/TMDB_movie_dataset_v11.csv", usecols=colunas_escolhidas)

In [5]:
print(f" Os seus dados estão com a seguinte configuração: {dados.shape}\n")
dados.head(2)

 Os seus dados estão com a seguinte configuração: (1249252, 13)



,id,title,vote_average,vote_count,release_date,revenue,runtime,adult,budget,overview,popularity,genres,keywords
0,27205,Inception,8.364,34495,2010-07-15,825532764,148,False,160000000,"Cobb, a skilled thief who commits corporate es...",83.952,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,2014-11-05,701729206,169,False,165000000,The adventures of a group of explorers who mak...,140.241,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,..."


In [5]:
colunas = list(dados.columns)
for coluna in colunas:
  print(coluna)

id
title
vote_average
vote_count
release_date
revenue
runtime
adult
budget
overview
popularity
genres
keywords


In [6]:
dados['keywords'].value_counts()

,count
keywords,
short film,10641
woman director,8357
gay pornography,8077
stand-up comedy,2588
concert,1982
...,...
"java, indonesia",1
"sports, amateur football (soccer), football (soccer), ireland, football (soccer) coach, football (soccer) player",1
"pre-code, aviator",1


In [7]:
dados.isnull().sum()

,0
id,0
title,13
vote_average,0
vote_count,0
release_date,236881
revenue,0
runtime,0
adult,0
budget,0
overview,268584


##Versão 1 do KNN

Utilizaremos as features mais diretas, principalmente as numéricas.

In [8]:
df_knn_v1 = dados

In [6]:
dados['release_date'] = pd.to_datetime(dados['release_date'])
dados['release_year'] = dados['release_date'].dt.year
dados = dados.drop('release_date', axis=1)
dados = dados.dropna(subset=['title'])
dados.dtypes

,0
id,int64
title,object
vote_average,float64
vote_count,int64
revenue,int64
runtime,int64
adult,bool
budget,int64
overview,object
popularity,float64


In [10]:
dados.head(2)

,id,title,vote_average,vote_count,revenue,runtime,adult,budget,overview,popularity,genres,keywords,release_year
0,27205,Inception,8.364,34495,825532764,148,False,160000000,"Cobb, a skilled thief who commits corporate es...",83.952,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc...",2010.0
1,157336,Interstellar,8.417,32571,701729206,169,False,165000000,The adventures of a group of explorers who mak...,140.241,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,...",2014.0


## Aplicando MinMax Scaller para normalizar os dados numéricos

In [7]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 1. Inicialize o scaler
scaler = MinMaxScaler()

# 2. Defina as colunas que você quer escalar
colunas_para_escalar = ['vote_average', 'vote_count', 'release_year', 'revenue', 'runtime', 'budget', 'popularity']

# 3. Use fit_transform nas colunas selecionadas
#    IMPORTANTE: O resultado do scaler é um array NumPy, não um DataFrame.
#    Por isso, o atribuímos de volta às colunas originais do DataFrame.
dados[colunas_para_escalar] = scaler.fit_transform(dados[colunas_para_escalar])

print("\nDataFrame com Variáveis Escaladas:")
display(dados.head(2))


DataFrame com Variáveis Escaladas:


,id,title,vote_average,vote_count,revenue,runtime,adult,budget,overview,popularity,genres,keywords,release_year
0,27205,Inception,0.8364,1.000000,0.165107,0.012199,False,0.160,"Cobb, a skilled thief who commits corporate es...",0.028037,"Action, Science Fiction, Adventure","rescue, mission, dream, airplane, paris, franc...",0.702341
1,157336,Interstellar,0.8417,0.944224,0.140346,0.013654,False,0.165,The adventures of a group of explorers who mak...,0.046835,"Adventure, Drama, Science Fiction","rescue, future, spacecraft, race against time,...",0.715719


In [8]:
#Transforma o booleano de Adult em 0 e 1
dados['adult'] = dados['adult'].astype(int)

#Imputando "Desconhecido" nos nulos de gênero
dados['genres'].fillna('Desconhecido', inplace=True)

/tmp/ipython-input-8-761902011.py:5: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  dados['genres'].fillna('Desconhecido', inplace=True)


In [ ]:
#OneHot Encoding dos gêneros
genres_dummies = pd.get_dummies(dados['genres'], prefix='genre')

# Agora, vamos juntar essas novas colunas ao nosso DataFrame original
# axis=1 significa que estamos concatenando colunas (lado a lado)
df_final = pd.concat([dados, genres_dummies], axis=1)

In [ ]:
df_final.head(2)

In [9]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.neighbors import NearestNeighbors
import pandas as pd

# Supondo que seu DataFrame se chama 'df'

# --- 1. Seleção de Features ---
# Vamos selecionar as colunas numéricas que já tratamos e consideramos úteis
colunas_numericas = ['vote_average', 'runtime', 'popularity', 'release_year', 'adult']

# Criamos um novo DataFrame apenas com elas, garantindo que não há NaNs
# (Assumindo que já fizemos a imputação da mediana ou removemos as linhas com nulos)
df_numerico = dados[colunas_numericas].dropna()

print("--- DataFrame com Features Numéricas ---")
print(df_numerico.head())

# --- 2. Escalonamento ---
# Todos os dados precisam estar na mesma escala para o KNN funcionar bem
scaler = MinMaxScaler()
features_escalonadas = scaler.fit_transform(df_numerico)

print("\n--- Amostra das Features Escalonadas (Array NumPy) ---")
print(features_escalonadas[:5])

# --- 3. Treinamento do Modelo ---
# n_neighbors=6 porque o primeiro vizinho (distância 0) será o próprio filme
# algorithm='brute' é direto, mas para datasets maiores 'kd_tree' ou 'ball_tree' podem ser mais rápidos
knn_model = NearestNeighbors(n_neighbors=6, algorithm='brute', metric='euclidean')

# "Treinamos" o modelo com nossos dados escalonados
knn_model.fit(features_escalonadas)

print("\nModelo KNN treinado com sucesso!")

--- DataFrame com Features Numéricas ---
   vote_average   runtime  popularity  release_year  adult
0        0.8364  0.012199    0.028037      0.702341      0
1        0.8417  0.013654    0.046835      0.715719      0
2        0.8512  0.012476    0.043630      0.695652      0
3        0.7573  0.013169    0.026694      0.698997      0
4        0.7710  0.011852    0.032756      0.709030      0

--- Amostra das Features Escalonadas (Array NumPy) ---
[[0.8364     0.01027778 0.02803674 0.70234114 0.        ]
 [0.8417     0.01173611 0.0468351  0.71571906 0.        ]
 [0.8512     0.01055556 0.04362973 0.69565217 0.        ]
 [0.7573     0.01125    0.02669421 0.69899666 0.        ]
 [0.771      0.00993056 0.03275561 0.7090301  0.        ]]

Modelo KNN treinado com sucesso!


In [14]:
import pandas as pd

# --- Pré-requisitos (código que já rodamos) ---
# Certifique-se de que estas variáveis já existem no seu ambiente:
# df: seu DataFrame original com a coluna 'title'.
# df_numerico: o DataFrame só com as colunas numéricas, sem NaNs.
# features_escalonadas: o array NumPy retornado pelo MinMaxScaler.
# knn_model: o objeto NearestNeighbors já treinado com .fit().

# Criamos um mapeamento do título do filme para o seu índice no DataFrame numérico.

# Usamos o índice do df_numerico para selecionar apenas os títulos correspondentes no df original
indices = pd.Series(df_numerico.index, index=dados.loc[df_numerico.index]['title'])

# --- A Função de Recomendação ---

def recomendar_filmes(titulo, modelo_knn, dados_escalonados, mapeamento_indices):
    """
    Recomenda 5 filmes similares a um filme de entrada com base em features numéricas.

    Args:
        titulo (str): O título do filme para o qual queremos recomendações.
        modelo_knn (NearestNeighbors): O modelo KNN já treinado.
        dados_escalonados (np.array): O array com todas as features escalonadas.
        mapeamento_indices (pd.Series): Um mapeamento de títulos para índices.

    Returns:
        list: Uma lista com os títulos dos 5 filmes mais recomendados.
    """
    # 1. Obter o índice do filme a partir do título
    try:
        idx = mapeamento_indices[titulo]
    except KeyError:
        return f"Filme '{titulo}' não encontrado no nosso banco de dados. Tente outro."

    # 2. Obter as distâncias e os índices dos 5 vizinhos mais próximos
    # O [idx] garante que estamos pegando o vetor de features daquele filme específico
    # kneighbors retorna duas coisas: as distâncias e os índices
    distancias, indices_vizinhos = modelo_knn.kneighbors([dados_escalonados[idx]])

    # 3. Os índices retornados são para o array 'dados_escalonados'.
    # O primeiro vizinho (índice 0) é o próprio filme, então pulamos ele (começamos do 1)
    indices_filmes_similares = indices_vizinhos[0][1:]

    # 4. Traduzir os índices de volta para títulos de filmes
    titulos_recomendados = dados['title'].iloc[indices_filmes_similares].tolist()

    return titulos_recomendados

# --- Exemplo de Uso ---
# Vamos pegar um filme famoso para testar
filme_exemplo = 'Jumanji: Welcome to the Jungle' # Ou qualquer outro filme que esteja no seu 'title'

recomendacoes = recomendar_filmes(filme_exemplo, knn_model, features_escalonadas, indices)

print(f"Porque você assistiu '{filme_exemplo}', talvez você goste de:")
for i, filme in enumerate(recomendacoes):
    print(f"{i+1}. {filme}")


Porque você assistiu 'Jumanji: Welcome to the Jungle', talvez você goste de:
1. The Man Who Invented Christmas
2. Mazinger Z: Infinity
3. Girls Trip
4. American Made
5. The Thinning
